# Connecting to Drive

In [1]:
# Connecting to google drive
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive
Mounted at /content/drive


In [2]:
# Path
%cd "/content/drive/MyDrive/Bridge construction health/brridge-failure-detection"

/content/drive/MyDrive/Bridge construction health/brridge-failure-detection
/content/drive/MyDrive/Bridge construction health/brridge-failure-detection


In [7]:
! ls -ltrh

total 4.7M
-rw------- 1 root root 868K Aug 19 15:51 data.rar
-rw------- 1 root root 3.4M Aug 19 15:51 data1.rar
-rw------- 1 root root 373K Aug 21 16:26 wavelet.ipynb
-rw------- 1 root root 3.0K Aug 21 16:26 sst_nino3.dat
-rw------- 1 root root  35K Aug 21 16:26 analysis.ipynb
drwx------ 2 root root 4.0K Aug 21 16:35 data
drwx------ 2 root root 4.0K Aug 21 16:36 data1
-rw------- 1 root root 8.2K Aug 21 18:38 github_handler.ipynb
drwx------ 2 root root 4.0K Aug 23 12:01 main_datatset
-rw------- 1 root root  50K Aug 30 18:29 bridge_failure_detection_mixed.ipynb
-rw------- 1 root root  47K Aug 30 18:33 bridge_failure_detection_one_sensor.ipynb


# Imports and installations

In [4]:
import os
import shutil
from pathlib import Path

In [5]:
from PIL import Image
import numpy as np

# Vectorization of images

In [6]:
def vectorize_rgb_image_to_array(png_path):
    # Load the PNG image
    image = Image.open(png_path)

    # Convert the image to RGB mode (if it's not already)
    image = image.convert('RGB')

    # Convert the image to a numpy array
    img_array = np.array(image)

    return img_array

In [18]:
! ls "./data1/data1/After_reconstruction" | head

1_After_reconstruction_0_1.png
1_After_reconstruction_0_2.png
1_After_reconstruction_0_3.png
1_After_reconstruction_0_4.png
1_After_reconstruction_100_1.png
1_After_reconstruction_100_2.png
1_After_reconstruction_100_3.png
1_After_reconstruction_100_4.png
1_After_reconstruction_101_1.png
1_After_reconstruction_101_2.png


## Checking the sample number of each class

In [15]:
len(os.listdir("./data1/data1/When_rebuilding"))

844

In [16]:
len(os.listdir("./data1/data1/Before_renovation"))

816

In [17]:
len(os.listdir("./data1/data1/After_reconstruction"))

844

## Vetorizing all

In [19]:
X = []
y = []

In [20]:
all_classes = os.listdir("./data1/data1/")

In [21]:
all_classes_dict = {
    "Before_renovation": 0,
    "After_reconstruction": 1,
    "When_rebuilding": 2
}

In [22]:
# Vectorizing and added labels of images
for lbl_class in all_classes:
  print(lbl_class)
  images = os.listdir(f"./data1/data1/{lbl_class}")

  for img in images:
    img_path = f"./data1/data1/{lbl_class}/{img}"
    img_array = vectorize_rgb_image_to_array(img_path)
    X.append(img_array)
    y.append(all_classes_dict[lbl_class])

When_rebuilding
Before_renovation
After_reconstruction


# Model Architecture

In [23]:
from sklearn.model_selection import train_test_split

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [24]:
X_train.shape

(2003, 23, 23, 3)

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(23, 23, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 21, 21, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85,187 (332.76 KB)

 Trainable params: 85,187 (332.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(23, 23, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

In [30]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.3494 - loss: 10.4526 - val_accuracy: 0.4431 - val_loss: 1.1949
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.4838 - loss: 1.1475 - val_accuracy: 0.5589 - val_loss: 0.9705
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5947 - loss: 0.8958 - val_accuracy: 0.5609 - val_loss: 0.9202
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6203 - loss: 0.8149 - val_accuracy: 0.5968 - val_loss: 0.9787
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.6511 - loss: 0.7791 - val_accuracy: 0.5788 - val_loss: 0.8683
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6480 - loss: 0.7787 - val_accuracy: 0.6627 - val_loss: 0.7459
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7059 - loss: 0.6667 - val_accuracy: 0.6487 - val_loss: 0.7302
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7478 - loss: 0.5930 - val_accuracy: 0.6507 - 

In [31]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7615 - loss: 1.3550
Test loss: 1.3806315660476685
Test accuracy: 0.7824351191520691
